In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime as DT
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense, BatchNormalization,Dropout
from tensorflow.keras import optimizers
import os
from matplotlib import pyplot as plt
from sklearn import svm, datasets
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
patients = pd.read_csv('./maindata/patients.csv')
allergies = pd.read_csv('./maindata/allergies.csv')

In [3]:
allergies = allergies[allergies.CATEGORY != 'medication']
print(len(allergies['CODE'].unique()))

15


In [4]:
AllergiesDetails = allergies[['CODE','DESCRIPTION','CATEGORY','REACTION1','DESCRIPTION1','SEVERITY1','REACTION2','DESCRIPTION2','SEVERITY2']]
AllergiesDetails = AllergiesDetails.drop_duplicates(subset='CODE',keep='last')

In [5]:
AllergiesDetails.to_csv('AllergiesList.csv',index=False)

In [6]:
allergies = allergies[['PATIENT','CODE','START']]

In [7]:
allergies = pd.merge(allergies,patients,how='left',left_on='PATIENT',right_on='Id')

In [8]:
allergies=allergies[['START','BIRTHDATE','LAT','LON','CODE']]
allergies['START'] = pd.to_datetime(allergies['START'], errors='coerce')
allergies['BIRTHDATE'] = pd.to_datetime(allergies['BIRTHDATE'], errors='coerce')

In [9]:
allergies['AGE']=allergies['BIRTHDATE']
allergies['AGE']=pd.to_numeric(allergies['AGE'].values)/10**9

In [10]:
allergies=allergies[['AGE','LAT','LON','CODE']]

In [11]:
allergies.to_csv('CleanedData.csv')

In [12]:
allergies.head()

,AGE,LAT,LON,CODE
0,1.507334e+09,41.930492,-71.003337,288328004
1,1.507334e+09,41.930492,-71.003337,84489001
2,1.507334e+09,41.930492,-71.003337,260147004
3,1.507334e+09,41.930492,-71.003337,264287008
4,1.507334e+09,41.930492,-71.003337,782576004


In [13]:
allergies_label = allergies.pop('CODE')

In [14]:
allergies_labels = pd.get_dummies(allergies_label,prefix="allergy")

In [15]:
allergies = pd.concat([allergies,allergies_labels],axis=1)

## Splitting Data in training and test

In [16]:
complete_train_dataset, complete_test_dataset = train_test_split(allergies,test_size=0.1)
# complete_test_dataset,complete_valid_dataset = train_test_split(temp_test_dataset,test_size=0.5)

In [17]:
train_dataset = complete_train_dataset.iloc[:,0:3]
train_labels = complete_train_dataset.iloc[:,3:]

In [18]:
test_dataset = complete_test_dataset.iloc[:,0:3]
test_labels = complete_test_dataset.iloc[:,3:]

#### Normalizing data

In [19]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()


In [76]:
list(train_dataset.iloc[0])

[789091200.0, 42.64548037467002, -71.39734343342467]

In [34]:
list(train_stats['std'])

[646655758.3289639, 0.2907192152409467, 0.6214539427248971]

In [20]:
def norm(x):
    return (x-train_stats['mean'])/train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)


In [40]:
list(normed_test_data.iloc[0])

[-2.1905921170725358, 0.08366228852026761, -0.144925575613581]

In [21]:
print(normed_test_data.shape[1])
print(train_labels.shape[1])

3
15


In [22]:

# model = Pipeline(steps=[('svr',SVR()),('lr_multi',MultiOutputClassifier(LogisticRegression()))])
# # model = MultiOutputRegressor(svr)
# model = model.fit(train_dataset,train_labels)
                        
# Create the SVR regressor
# svr = SVR(epsilon=0.2)

# Create the Multioutput Regressor
# model = MultiOutputRegressor(svr)

model = Pipeline([('reg', MultiOutputRegressor(SVR()))])


grid_param_svr = {
    'reg__estimator__C': [0.01,0.1,1],
    'reg__estimator__epsilon':[0.1,0.2],
    'reg__estimator__degree':[2,3]
}

gs_svr = (GridSearchCV( estimator=model, 
                        param_grid=grid_param_svr, 
                        cv=20,
                        scoring = 'accuracy',
                        n_jobs = -1,
                        refit=True)
         )

gs_svr = gs_svr.fit(train_dataset,train_labels)


/opt/anaconda3/envs/convo/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/convo/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/convo/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/opt/anaconda3/envs/convo/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/opt/anaconda3/envs/convo/lib/python3.7/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/opt/anaconda3/envs/convo/lib/python3.7/site-packages

In [26]:
example_batch = normed_test_data
example_result = gs_svr.predict(example_batch)
example_result

array([[0.0999913 , 0.10006935, 0.10009937, ..., 0.10001634, 0.10002487,
        0.09999817],
       [0.0999913 , 0.10006935, 0.10009937, ..., 0.10001634, 0.10002487,
        0.09999817],
       [0.0999913 , 0.10006935, 0.10009937, ..., 0.10001634, 0.10002487,
        0.09999817],
       ...,
       [0.0999913 , 0.10006935, 0.10009937, ..., 0.10001634, 0.10002487,
        0.09999817],
       [0.0999913 , 0.10006935, 0.10009937, ..., 0.10001634, 0.10002487,
        0.09999817],
       [0.0999913 , 0.10006935, 0.10009937, ..., 0.10001634, 0.10002487,
        0.09999817]])

In [27]:
import joblib

joblib.dump(gs_svr, "./geolocationAllergyPrediction.joblib")

['./geolocationAllergyPrediction.joblib']

In [43]:
# gs_svr.score(normed_test_data,test_labels)

In [28]:
# Evaluate the regressor
mse_one = mean_squared_error(test_labels.iloc[:,0], example_result[:,0])
mse_two = mean_squared_error(test_labels.iloc[:,1], example_result[:,1])
print(f'MSE for first regressor: {mse_one} - second regressor: {mse_two}')
mae_one = mean_absolute_error(test_labels.iloc[:,0], example_result[:,0])
mae_two = mean_absolute_error(test_labels.iloc[:,1], example_result[:,1])
print(f'MAE for first regressor: {mae_one} - second regressor: {mae_two}')

MSE for first regressor: 0.036245550286522606 - second regressor: 0.12547941643942195
MAE for first regressor: 0.12623859130987103 - second regressor: 0.21553489492382258


In [29]:
index = 0
correct = 0
incorrect = 0
total = 0
for i in example_result:
    temp = np.array(example_result[index])
    predicted = temp.argsort()[-5:]
    actual = np.argmax(test_labels.iloc[index])
    if actual in predicted:
        correct+=1
    else:
        incorrect+=1
    total+=1
    index+=1
print(correct/total)
# np.argmax(example_result[1])

0.44750656167979


In [48]:
def norm2(x):
    res = [[
        x[0][0]-635589406.6549913/646655758.3289639,
        x[0][1]-42.25797846733888/0.2907192152409467,
        x[0][2]-71.36106653596275/0.6214539427248971
    ]]
    return res


In [37]:
loaded_rf = joblib.load("../models/geolocationAllergyPrediction.joblib")


In [49]:
data = [[-2.1905921170725358, 0.08366228852026761, -0.144925575613581]]
scaled_data = norm2(data)
scaled_data

[[-3.1734789134376227, -145.2730126471756, -114.97413757331202]]

In [53]:
predicted = loaded_rf.predict(scaled_data)
predicted

/opt/anaconda3/envs/convo/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  "X does not have valid feature names, but"
/opt/anaconda3/envs/convo/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  "X does not have valid feature names, but"
/opt/anaconda3/envs/convo/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  "X does not have valid feature names, but"
/opt/anaconda3/envs/convo/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  "X does not have valid feature names, but"
/opt/anaconda3/envs/convo/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  "X 

array([[0.0999913 , 0.10006935, 0.10009937, 0.10001343, 0.10001399,
        0.09989847, 0.10000947, 0.10013622, 0.10000375, 0.10006173,
        0.10009196, 0.09995628, 0.10001634, 0.10002487, 0.09999817]])

In [54]:
predicted = np.array(predicted[0]).argsort()[-5:]
predicted

array([ 9,  1, 10,  2,  7])

In [61]:
allergy_codes = [s.split('_')[1] for s in list(allergies_labels.columns)]

In [65]:
allergy_ids = [allergy_codes[i] for i in predicted]

In [73]:
allergy_details = pd.read_csv('../data/AllergiesList.csv', index_col='CODE',dtype={'CODE':'string'})
allergy_details.head()

,DESCRIPTION,CATEGORY,REACTION1,DESCRIPTION1,SEVERITY1,REACTION2,DESCRIPTION2,SEVERITY2
CODE,,,,,,,,
256355007,Soya bean (substance),food,62315008.0,Diarrhea (finding),MILD,NaN,NaN,NaN
3718001,Cow's milk (substance),food,247472004.0,Wheal (finding),MILD,NaN,NaN,NaN
288328004,Bee venom (substance),environment,NaN,NaN,NaN,NaN,NaN,NaN
111088007,Latex (substance),environment,267036007.0,Dyspnea (finding),MODERATE,NaN,NaN,NaN
735029006,Shellfish (substance),food,39579001.0,Anaphylaxis (disorder),SEVERE,271807003.0,Eruption of skin (disorder),MODERATE


In [74]:
res = []
for code in allergy_ids:
    res.append(allergy_details.loc[code].to_dict())

res


[{'DESCRIPTION': 'Wheat (substance)',
  'CATEGORY': 'food',
  'REACTION1': nan,
  'DESCRIPTION1': nan,
  'SEVERITY1': nan,
  'REACTION2': nan,
  'DESCRIPTION2': nan,
  'SEVERITY2': nan},
 {'DESCRIPTION': 'Mold (organism)',
  'CATEGORY': 'environment',
  'REACTION1': 267101005.0,
  'DESCRIPTION1': 'Nose running',
  'SEVERITY1': 'MILD',
  'REACTION2': nan,
  'DESCRIPTION2': nan,
  'SEVERITY2': nan},
 {'DESCRIPTION': 'Shellfish (substance)',
  'CATEGORY': 'food',
  'REACTION1': 39579001.0,
  'DESCRIPTION1': 'Anaphylaxis (disorder)',
  'SEVERITY1': 'SEVERE',
  'REACTION2': 271807003.0,
  'DESCRIPTION2': 'Eruption of skin (disorder)',
  'SEVERITY2': 'MODERATE'},
 {'DESCRIPTION': 'Eggs (edible) (substance)',
  'CATEGORY': 'food',
  'REACTION1': nan,
  'DESCRIPTION1': nan,
  'SEVERITY1': nan,
  'REACTION2': nan,
  'DESCRIPTION2': nan,
  'SEVERITY2': nan},
 {'DESCRIPTION': 'Animal dander (substance)',
  'CATEGORY': 'environment',
  'REACTION1': 878820003.0,
  'DESCRIPTION1': 'Rhinoconjunctivit